In [7]:
using DrWatson
@quickactivate "EIT"
using Revise, ProgressMeter, Distributions, Random
includet(projectdir("config", "example_config.jl"))
includet(srcdir("funcs.jl"))
import ..Simu
using ..MoMForward: ConstantParameter
import ..MoMForward as mf


In [3]:
config

Dict{String, Real} with 12 entries:
  "dis_tx_rx"     => 8.0
  "doi_size_x"    => 5.0
  "tag_size"      => 0.2
  "grids_x"       => 500
  "dis_target_rx" => 3.0
  "rx_num"        => 3
  "N"             => 100
  "grids_y"       => 1000
  "tx_num"        => 1
  "doi_size_y"    => 10.0
  "freq"          => 5.0e9
  "dis_antenna"   => 0.03

In [4]:
function get_scatterer(; grid_xs, grid_ys, center, r, ϵ)
    scatterer = ones(Complex, length(grid_ys), length(grid_xs))
    for i in eachindex(grid_xs), j in eachindex(grid_ys)
        if (grid_xs[i] - center[1])^2 + (grid_ys[j] - center[2])^2 < r^2
            scatterer[j, i] = ϵ
        end
    end
    return scatterer
end

get_scatterer (generic function with 1 method)

In [8]:
function get_E_in_rx_dis_tag_rx(config_now)
    @unpack freq, grids_x, grids_y, doi_size_x, doi_size_y, tx_num, rx_num, dis_antenna, tag_size, dis_tx_rx, dis_target_rx,N = config_now

    txs_pos_x = repeat([doi_size_x / 2 - dis_tx_rx], tx_num)'
    txs_pos_y = ((collect(1:tx_num) .- (tx_num + 1) / 2) .* dis_antenna)'
    txs_pos = [txs_pos_x; txs_pos_y] # 2 x tx_num
    rxs_pos_x = repeat([doi_size_x / 2], rx_num)'
    rxs_pos_y = ((collect(1:rx_num) .- (rx_num + 1) / 2) .* dis_antenna)'
    rxs_pos = [rxs_pos_x; rxs_pos_y] # 2 x rx_num
    parameters = mf.ConstantParameter(
        freq = freq, doi_size_x = doi_size_x, doi_size_y = doi_size_y, grid_number_x = grids_x,
        grid_number_y = grids_y, txs_pos = txs_pos, rxs_pos = rxs_pos
    )
    shared_variables, _ = produce_or_load(
        Simu.sim_share_variables, parameters, datadir("example","shared_variables"); verbose = false)
    ϵ = 10.0 - 2.0im
    tag_radio = tag_size

    AoA = rand(Uniform(0, π), N) .+ pi / 2
    tag_pos = [cos.(AoA) .* dis_target_rx sin.(AoA) .* dis_target_rx]
    Ei_all = Vector{Array{ComplexF64}}(undef, N)
    Es_all = Vector{Array{ComplexF64}}(undef, N)
    Ray_all = Vector{Array{ComplexF64}}(undef, N)
    Threads.@threads for i in 1:N
        tag_pos_now = tag_pos[i, :]
        temp_scatterer = get_scatterer(
            grid_xs = parameters.grid_xs, grid_ys = parameters.grid_ys,
            center = tag_pos_now, r = tag_radio, ϵ = ϵ)
        Es_all[i] = Simu.get_Es(parameters, shared_variables, temp_scatterer)#The size is [rx_num, tx_num]
        Ei_all[i] = shared_variables["Ei_in_rxs"]#The size is [rx_num, tx_num]
    end
    return @strdict Ei_all Es_all  AoA
end

get_E_in_rx_dis_tag_rx (generic function with 1 method)

In [9]:
data, _ = produce_or_load(
            x -> get_E_in_rx_dis_tag_rx(x), config, datadir("example"); verbose = false
        )

(Dict{String, Vector}("AoA" => [3.7285318695657406, 3.5770749768505623, 4.016432043161725, 2.7202767247630937, 3.381836143550185, 2.288387476252896, 4.507168670775053, 4.129417310687078, 2.2837081917018858, 3.5679408163748536  …  3.305487062538301, 3.0856639819694758, 4.449618328457731, 2.8901263468913854, 4.246481192896978, 4.639222412435204, 2.5482504201740603, 3.237549944302287, 3.732386031263241, 3.3025953536335715], "Es_all" => Array{ComplexF64}[[0.03639634477069132 - 0.01605282586356414im; 0.03280836444389934 + 0.023517121661127238im; -0.00420926072677421 + 0.04077369646970601im;;], [0.0 + 0.0im; 0.0 + 0.0im; 0.0 + 0.0im;;], [-0.0004805516999164053 - 0.023000258246188485im; 0.02277452406003188 - 0.0018698796380012261im; 0.002911165714802995 + 0.022524536491315323im;;], [0.0 + 0.0im; 0.0 + 0.0im; 0.0 + 0.0im;;], [0.0 + 0.0im; 0.0 + 0.0im; 0.0 + 0.0im;;], [-0.012737789357368613 - 0.027241735682425036im; -0.028640288472078963 + 0.010522710241853458im; 0.007718368160863669 + 0.029983